In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
import os
import scipy
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img

2024-05-20 22:27:57.242761: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 22:27:57.248340: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 22:27:57.312602: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 22:27:58.697240: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def augment_images(directories, datagen, num_augmented_images=5) -> None:
    for country in os.listdir(directories['input_dir']):
        country_input_path = os.path.join(directories['input_dir'], country)
        country_output_path = os.path.join(directories['output_dir'], country)
        os.makedirs(country_output_path, exist_ok=True)

        for image_name in os.listdir(country_input_path):
            img_path = os.path.join(country_input_path, image_name)
            img = load_img(img_path)
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)
            # to size: (1, width, height, channels)
            
            i = 0
            for batch in datagen.flow(x, batch_size=1, save_to_dir=country_output_path, save_prefix='aug', save_format='jpg'):
                i += 1
                if i >= num_augmented_images:
                    break

In [3]:
# obj for augmentation:
datagen = ImageDataGenerator(
    rotation_range=90.0,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.4,
    zoom_range=0.5,
    channel_shift_range=90.0,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
dir_dict = {
    'input_dir': 'data/country_flag',
    'output_dir': 'data/augmented_flags'
}
os.makedirs(dir_dict['output_dir'], exist_ok=True)

In [5]:
augment_images(dir_dict, datagen, num_augmented_images=200)
print("Augmentation completed.")

Augmentation completed.
